# Detective Story Experiment 
8.27.24

In [1]:
import os
import transformerModelFunctions
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import textwrap
wrapper = textwrap.TextWrapper(width=70)
tf.keras.utils.set_random_seed(10)
import glob
from collections import Counter
from collections import defaultdict
import re
import random

In [15]:
!pip uninstall keras

Found existing installation: keras 3.5.0
Uninstalling keras-3.5.0:
  Would remove:
    /Applications/anaconda3/lib/python3.9/site-packages/keras-3.5.0.dist-info/*
    /Applications/anaconda3/lib/python3.9/site-packages/keras/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [ ]:
install.packages('keras3')

In [2]:
fname = "~/Documents/GitHub/BMDS/BMDS_story_annotations.csv"

In [3]:
metaDF = pd.read_csv(fname)

In [4]:
metaList =[]
for index, row in metaDF.iterrows(): 
    metaList.append({'code': row['Story Code'], 'title': row['Story Title'], 'author': row['Author Code'], 'key clue': row['Essential clue'], 
                     'structure': row['Investigation-reveal order'], 'reveal line': row['Reveal border sentence']})

In [48]:
print(Counter(metaDF['Sufficient clues to solve?']))
print(Counter(metaDF['Correct annotator guess?']))

Counter({'No': 385, 'Yes': 35, nan: 15})
Counter({'No': 264, 'Yes': 156, nan: 15})


In [5]:
files = glob.glob("/Users/collinjennings/Documents/GitHub/BMDS/texts/*.txt")
texts = defaultdict()

for afile in files: 
    texts[re.sub('_','', afile.split('.')[0].split('/')[7])] = open(afile, encoding = 'utf-8').read()

In [27]:
Counter(metaDF['Sufficient clues to guess?']) #['Sufficient Clues to Guess?']

Counter({'Yes': 250, 'No': 170, nan: 15})

In [6]:
clueTextList = [] 
for item in metaList:
    if item['key clue'] != '_none' and item['code'] in texts.keys(): 
        item['text'] = '[SOS]'+texts[item['code']]+'[EOS]'
        item['key clue'] = '[SOS]'+item['key clue']+'[EOS]'
        clueTextList.append(item)

In [28]:
#clueTextList[1]

In [23]:
len(clueTextList)

413

In [25]:
#print(clueTextList[0])

## Steps
1. Organize the metadata as a list of dictionaries. 
2. Process texts as lists of sentences stored as word lists. 
3. Create lists of essential clues and pair with appropriate story texts. 
4. Match the "essential clue" with a specific appearance in the story full text. 
5. Mark the sentence in which the reveal begins. 

I think I want to run 2 versions of the experiment: one in which I give the classifier the full story, and one in which I make it rely on the story up to the point of the reveal. I would expect that in the full story, the classifier can look for the most repeated words/phrases to identify the key clue. 

**Designing the Experiment**
I'm trying to think of a model for this kind of experiment since I'm not doing a two or multi class experiment. I'm asking the model to find the same kind of term/phrase in each story. 


## Creating the Detective Model

Here I'm thinking about the "essential clue" as a kind of summary about the document.

Tweaks: 
- add [SOS] and [EOS] to text and clues. 

In [7]:
split = int(len(clueTextList)*.8)
trainList = clueTextList[:split]
testList = clueTextList[split:]
#random.shuffle(trainList)
#random.shuffle(testList)
#print(trainList[-1])
#print(testList[0])

In [8]:
train_data = pd.DataFrame.from_records(trainList, columns=['key clue', 'text'])
test_data = pd.DataFrame.from_records(testList, columns=['key clue', 'text'])

In [30]:
dectDF.iloc[1]

key clue    [SOS]The handwriting samples demanded by the c...
text        [SOS]\nShortly after my marriage I had bought ...
Name: 1, dtype: object

In [ ]:
## Modify to use my created corpus

# Take one example from the dataset and print it
example_summary, example_dialogue = train_data.iloc[10]
print(f"Dialogue:\n{example_dialogue}")
print(f"\nSummary:\n{example_summary}")

In [ ]:
train_data = de

In [9]:
### Need to input data as strings. 
document, summary = transformerModelFunctions.preprocess(train_data)
document_test, summary_test = transformerModelFunctions.preprocess(test_data)

In [10]:
# The [ and ] from default tokens cannot be removed, because they mark the SOS and EOS token.
## Clue descriptions will replace `summary` here. 

filters = '!"#$%&()*+,-./:;<=>?@\\^_`{|}~\t\n'
oov_token = '[UNK]'

tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token, lower=False)

documents_and_summary = pd.concat([document, summary], ignore_index=True)

tokenizer.fit_on_texts(documents_and_summary)

inputs = tokenizer.texts_to_sequences(document)
targets = tokenizer.texts_to_sequences(summary)

vocab_size = len(tokenizer.word_index) + 1

print(f'Size of vocabulary: {vocab_size}')

Size of vocabulary: 53552


In [17]:
# Limit the size of the input and output data for being able to run it in this environment.
encoder_maxlen = 150
decoder_maxlen = 50

# Pad the sequences.
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')

inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

# Create the final training dataset.
BUFFER_SIZE = 10000
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [12]:
# Define the model parameters
num_layers = 2
embedding_dim = 128
fully_connected_dim = 128
num_heads = 2
positional_encoding_length = 256

# Initialize the model
transformer = transformerModelFunctions.Transformer(
    num_layers, 
    embedding_dim, 
    num_heads, 
    fully_connected_dim,
    vocab_size, 
    vocab_size, 
    positional_encoding_length, 
    positional_encoding_length,
)

learning_rate = transformerModelFunctions.CustomSchedule(embedding_dim)
optimizer = tf.keras.optimizers.Adam(0.0002, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
train_loss = tf.keras.metrics.Mean(name='train_loss')
# Here you will store the losses, so you can later plot them
losses = []


## Train the model

In [19]:
# Take an example from the test set, to monitor it during training
test_example = 0
true_summary = summary_test[test_example]
true_document = document_test[test_example]

# Define the number of epochs
epochs = 20

# Training loop
for epoch in range(epochs):
    
    start = time.time()
    train_loss.reset_state()
    number_of_batches=len(list(enumerate(dataset)))

    for (batch, (inp, tar)) in enumerate(dataset):
        print(f'Epoch {epoch+1}, Batch {batch+1}/{number_of_batches}', end='\r')
        transformerModelFunctions.train_step(transformer, inp, tar)
    
    print (f'Epoch {epoch+1}, Loss {train_loss.result():.4f}')
    losses.append(train_loss.result())
    
    print (f'Time taken for one epoch: {time.time() - start} sec')
    print('Example summarization on the test set:')
    print('  True summarization:')
    print(f'    {true_summary}')
    print('  Predicted summarization:')
    print(f'    {summarize(transformer, true_document)}\n')

ValueError: in user code:

    File "/Users/collinjennings/Dropbox/NJMove/detectiveStories/transformerModelFunctions.py", line 533, in train_step  *
        predictions, _ = model(
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/Applications/anaconda3/lib/python3.9/site-packages/keras/src/layers/layer.py", line 808, in __call__
        raise ValueError(

    ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: True (of type <class 'bool'>)


In [ ]:
plt.plot(losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

## Solve the stories!

In [ ]:
training_set_example = 0

# Check a summary of a document from the training set
print('Training set example:')
print(document[training_set_example])
print('\nHuman written summary:')
print(summary[training_set_example])
print('\nModel written summary:')
print(summarize(transformer, document[training_set_example]))

In [ ]:
test_set_example = 3

# Check a summary of a document from the test set
print('Test set example:')
print(document_test[test_set_example])
print('\nHuman written summary:')
print(summary_test[test_set_example])
print('\nModel written summary:')
print(summarize(transformer, document_test[test_set_example]))

## Measure the accuracy of the model